## 이미지 집합 끼리의 특징 거리를 계산해주는 FID를 사용하여 사고와 도로의 상관관계를 파악

- 전체 도로 이미지 집합의 분포에서 추출한 사고 O 도로와 사고 X 도로
- 이는 '사고'의 여부를 조건으로 추출된 것.
- 만약 '사고'라는 조건으로 수집한 이미지 집합의 분포가 전체 도로 이미지 집합 분포와 거리가 크다면, 이는 사고라는 요인이 도로 이미지와 상관관계가 있음을 의미한다고 볼 수 있음
- 또한, (사고 O 와 사고 X 도로)의 FID 는  (사고 O 와 전체 도로) FID 보다 더욱 더 클 가능성이 있음 (전체 도로는 사고O를 포함하고 있기 때문)

In [1]:
!pip install clean-fid

  Using cached clean_fid-0.1.32-py3-none-any.whl (26 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Attempting uninstall: idna
    Found existing installation: idna 3.3
    Uninstalling idna-3.3:
      Successfully uninstalled idna-3.3
  Attempting uninstall: requests
    Found existing installation: requests 2.28.0
    Uninstalling requests-2.28.0:
      Successfully uninstalled requests-2.28.0


In [3]:
from cleanfid import fid

dir1 =  '/data/accident/images/'
dir2 = '/data/random/images/'
dir3 = '/data/no_accident/images/'
no_accident_80 = '/data/no_accident_80_500/images/'
random_80 = '/data/random_80/images/'
accident_80 =  '/data/accident_80/images/'

random_50 = '/data/random_50/images/'
accident_50 =  '/data/accident_50/images/'
no_accident_50 = '/data/no_accident_50_500/images/'

## FID를 계산할 10000개의 데이터를 샘플링

In [4]:
dir_list = [random_50, no_accident_50, accident_50]

def make_dirs(dirs):
    if not os.path.exists(dirs):
        os.makedirs(dirs)
tmp_dirname = './tmp_data'
# random folder
count = 0
for dir_path  in dir_list:
    make_dirs(tmp_dirname+dir_path)

    from pathlib import Path
    import random

    pathlist = Path('./'+dir_path).glob('**/*.png')
    nof_samples = 9000
    rc = []
    rc = []
    for k, path in enumerate(pathlist):
        if k < nof_samples:
            rc.append(str(path)) # because path is object not string
        else:
            i = random.randint(0, k)
            if i < nof_samples:
                rc[i] = str(path)

    import shutil            
    idx = 0
    for src in rc:
        dst = tmp_dirname +  dir_path + str(idx) + '.png'
        shutil.copyfile(src, dst)
        idx+=1

In [5]:
tmp_random_80 = tmp_dirname +  random_50
tmp_accident_80 = tmp_dirname +  accident_50
tmp_no_accident_80 = tmp_dirname +  no_accident_50

## 전체 도로와 사고 O 도로

In [6]:
score = fid.compute_fid(tmp_random_80, tmp_accident_80, num_workers=0)
print(score)

compute FID between two folders
Found 18000 images in the folder ./tmp_data/data/random_50/images/


FID  : 100%|██████████| 563/563 [04:00<00:00,  2.34it/s]


Found 18000 images in the folder ./tmp_data/data/accident_50/images/


FID  : 100%|██████████| 563/563 [03:40<00:00,  2.55it/s]


26.534746132950403


## 사고 X 도로와 전체 도로의 FID

In [7]:
score = fid.compute_fid(tmp_random_80, tmp_no_accident_80, num_workers=0)
print(score)

compute FID between two folders
Found 18000 images in the folder ./tmp_data/data/random_50/images/


FID  : 100%|██████████| 563/563 [03:12<00:00,  2.92it/s]


Found 18000 images in the folder ./tmp_data/data/no_accident_50_500/images/


FID  : 100%|██████████| 563/563 [03:48<00:00,  2.46it/s]


2.3097603610657416


## 사고 O 도로와 사고 X 도로

In [8]:
score = fid.compute_fid(tmp_accident_80, tmp_no_accident_80, num_workers=0)
print(score)

compute FID between two folders
Found 18000 images in the folder ./tmp_data/data/accident_50/images/


FID  : 100%|██████████| 563/563 [03:11<00:00,  2.94it/s]


Found 18000 images in the folder ./tmp_data/data/no_accident_50_500/images/


FID  : 100%|██████████| 563/563 [03:10<00:00,  2.95it/s]


32.00177441244119


## 전체 도로 이미지와 전체 도로 이미지를 위한 샘플링

In [22]:
random_80 = '/data/random_80/images'

dir_list = [random_80 ] * 10

def make_dirs(dirs):
    if not os.path.exists(dirs):
        os.makedirs(dirs)
tmp_dirname = './tmp_data'
# random folder
count = 0
for dir_path  in dir_list:
    new_dir = tmp_dirname+dir_path + str(count)
    count +=1
    make_dirs(new_dir)

    from pathlib import Path
    import random

    pathlist = Path('./'+dir_path).glob('**/*.png')
    nof_samples = 10000
    rc = []
    rc = []
    for k, path in enumerate(pathlist):
        if k < nof_samples:
            rc.append(str(path)) # because path is object not string
        else:
            i = random.randint(0, k)
            if i < nof_samples:
                rc[i] = str(path)

    import shutil            
    idx = 0
    for src in rc:
        dst = new_dir + '/' + str(idx) + '.png'
        shutil.copyfile(src, dst)
        idx+=1

## 전체 도로 20000 개의 이미지에서 10000개의 이미지를 10번 샘플링하여 서로의 FID를 계산

- 결과값은 0~1

In [25]:
path  = 'D:\\1_SW2\\tmp_data\\data\\random_80\\images'
fid_list =[[0]*10]*10

for i in range(10):
    for j in range(10):
        fid_list[i][j] = fid.compute_fid(path+str(i), path+str(j), num_workers=0)
        print(i, j, fid_list[i][j])

compute FID between two folders
Found 20000 images in the folder D:\1_SW2\tmp_data\data\random_80\images0


FID images0 : 100%|██████████| 625/625 [04:16<00:00,  2.44it/s]


Found 20000 images in the folder D:\1_SW2\tmp_data\data\random_80\images0


FID images0 : 100%|██████████| 625/625 [03:42<00:00,  2.81it/s]


0 0 -2.0406787371030077e-11
compute FID between two folders
Found 20000 images in the folder D:\1_SW2\tmp_data\data\random_80\images0


...

FID images9 : 100%|██████████| 625/625 [03:30<00:00,  2.97it/s]


3 9 1.108213520016136
compute FID between two folders
Found 20000 images in the folder D:\1_SW2\tmp_data\data\random_80\images4


FID images4 : 100%|██████████| 625/625 [03:35<00:00,  2.91it/s]


Found 20000 images in the folder D:\1_SW2\tmp_data\data\random_80\images0


FID images0 : 100%|██████████| 625/625 [03:29<00:00,  2.98it/s]


In [ ]:
print(fid_list)